LASSO

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Linear_ed
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from log_linear_binary import E_lasso_bin
from optITR import optITR_lin_bined
import time
#As an example for linear case
# patiences = 10
# n_node = 50
# n_layer = 1
# t_nodes = 100
# t_fig = 50
# n_lr = 1e-3
# n_epoch = 1000
# s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')    
    val_data = gendata_Linear_ed(400, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(1600, 0.5,beta,gamma)
    St = E_lasso_bin(train_data, val_data, test_data,lam_min=-2,lam_max=0,K=5)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bined(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.04037000000000001
0.013064574237226408
0.001333081163293921
0.0008449477582590368


In [3]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Deep_ed
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from log_linear_binary import E_lasso_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Deep_ed(400, 0.5)
    train_data = gendata_Deep_ed(1600, 0.5)
    St = E_lasso_bin(train_data, val_data, test_data,lam_min=-2,lam_max=0,K=5)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.31204
0.010744226356513538
0.09893062923876478
0.008964187632043579


RDL

In [4]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Linear_ed
from log_dlearn_binary import RDL_bin
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')    
    val_data = gendata_Linear_ed(400, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(1600, 0.5,beta,gamma)
    res_rdl = RDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_rdl['itr_test']
    X_test = test_data['X']
    d_opt = X_test[:,0:5]@(gamma-beta)>0 
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    #quality value 
    theta = np.vstack((beta,gamma))
    quality = X_test[:,0:5]@theta.T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.05447000000000001
0.016067330207598277
0.003237726755026157
0.0026628782397643243


In [5]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Deep_ed
from log_dlearn_binary import RDL_bin
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Deep_ed(400, 0.5)
    train_data = gendata_Deep_ed(1600, 0.5)
    res_rdl = RDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_rdl['itr_test']
    X = test_data['X']
    g_X = test_data['g_X']
    h_X = test_data['h_X']
    # g_X =  np.cos(X[:,0]**2+2*X[:,1]**2+X[:,2]**3+np.sqrt(X[:,3]+1)*np.log(X[:,4]+2)/20)
        #A=1, influence h_X
    # h_X = X[:,0]/3 + np.exp(X[:,1])/4 + np.cos(X[:,2]* X[:,3])\
                # - (np.log(X[:,4]+2))/3 -1
    d_opt = (h_X-g_X)>0
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))

    
    quality = (np.vstack((g_X , h_X))).T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.3154
0.012630122723077564
0.10247316622087763
0.009172453018156728


SDL

In [6]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Linear_ed
from log_dlearn_binary import SDL_bin
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear_ed(400, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(1600, 0.5,beta,gamma)
    res_sdl = SDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X_test = test_data['X']
    d_opt = X_test[:,0:5]@(gamma-beta)>0 
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    #quality value 
    theta = np.vstack((beta,gamma))
    quality = X_test[:,0:5]@theta.T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.07742
0.023153047315634286
0.00805767516044361
0.005929774117299405


In [7]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Deep_ed 
from log_dlearn_binary import SDL_bin
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')  
    val_data = gendata_Deep_ed(400, 0.5)
    train_data = gendata_Deep_ed(1600, 0.5)
    res_sdl = SDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X = test_data['X']
    g_X = test_data['g_X']
    h_X = test_data['h_X']
    # g_X =  np.cos(X[:,0]**2+2*X[:,1]**2+X[:,2]**3+np.sqrt(X[:,3]+1)*np.log(X[:,4]+2)/20)
        #A=1, influence h_X
    # h_X = X[:,0]/3 + np.exp(X[:,1])/4 + np.cos(X[:,2]* X[:,3])\
                # - (np.log(X[:,4]+2))/3 -1
    d_opt = (h_X-g_X)>0
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    
    quality = (np.vstack((g_X , h_X))).T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    Regret = V_opt - V_est
    regret[i] = Regret
    # regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.30268999999999996
0.01006697074595929
0.09361690774513433
0.009441606348778409


DL

In [9]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Linear_ed
from log_dlearn_binary import DL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear_ed(400, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(1600, 0.5,beta,gamma)
    res_sdl = DL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X_test = test_data['X']
    d_opt = X_test[:,0:5]@(gamma-beta)>0 
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    #quality value 
    theta = np.vstack((beta,gamma))
    quality = X_test[:,0:5]@theta.T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.06644000000000001
0.019969136185624056
0.005527809955944162
0.004561953667611873


In [10]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Deep_ed 
from log_dlearn_binary import DL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')  
    val_data = gendata_Deep_ed(400, 0.5)
    train_data = gendata_Deep_ed(1600, 0.5)
    res_sdl = DL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X = test_data['X']
    g_X = test_data['g_X']
    h_X = test_data['h_X']
    # g_X =  np.cos(X[:,0]**2+2*X[:,1]**2+X[:,2]**3+np.sqrt(X[:,3]+1)*np.log(X[:,4]+2)/20)
        #A=1, influence h_X
    # h_X = X[:,0]/3 + np.exp(X[:,1])/4 + np.cos(X[:,2]* X[:,3])\
                # - (np.log(X[:,4]+2))/3 -1
    d_opt = (h_X-g_X)>0
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    
    quality = (np.vstack((g_X , h_X))).T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    Regret = V_opt - V_est
    regret[i] = Regret
    # regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.30781
0.010778863576462976
0.09759111742548768
0.008883661086813047


OLS

In [11]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Linear_ed
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from optITR import optITR_lin_bined
import time
#As an example for linear case
# patiences = 10
# n_node = 50
# n_layer = 1
# t_nodes = 100
# t_fig = 50
# n_lr = 1e-3
# n_epoch = 1000
# s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear_ed(400, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(1600, 0.5,beta,gamma)
    St = E_ols_bin(train_data, val_data, test_data)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bined(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.05757000000000002
0.013815755498705092
0.004145543521328974
0.0027243682403603926


In [12]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Deep_ed
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')
    val_data = gendata_Deep_ed(400, 0.5)
    train_data = gendata_Deep_ed(1600, 0.5)
    St = E_ols_bin(train_data, val_data, test_data)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.31427999999999995
0.011913085242706872
0.09982486030775398
0.008857883372036671


DLS

In [13]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Linear_ed
import torch
from log_dls_binary import E_dls_bin
from optITR import optITR_lin_bined
from optITR import optITR_deep_bin
import time
loop = 200
patiences = 20
n_node = 64
n_layer = 2
n_lr = 1e-3
n_epoch = 1000
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Linear_ed(400, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(1600, 0.5,beta,gamma)
    St = E_dls_bin(train_data, val_data, test_data, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bined(test_data,Cmean_R_X_treated, Cmean_R_X_untreat,beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.08731
0.01784779818352953
0.009621917428943364
0.00435771264870566


In [42]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Deep_ed
import torch
from log_dls_binary import E_dls_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200
patiences = 10
n_node = 64
n_layer = 1
n_lr = 3e-3
n_epoch = 1000
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Deep_ed(400, 0.5)
    train_data = gendata_Deep_ed(1600, 0.5)
    St = E_dls_bin(train_data, val_data, test_data, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.14908
0.04653293027523627
0.03353486304761659
0.02172656292263452


Proposed

In [15]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_ed
from data_generator import gendata_Deep_hetero
import torch
from log_hazard_binary import g_dnn_bin
from optITR import optITR_lin_bined
import time
#As an example for linear case
patiences = 30
n_node = 64
n_layer = 1
t_nodes = 100
t_fig = 50
n_lr = 1e-4
n_epoch = 1000
s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear_ed(400, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(1600, 0.5,beta,gamma)
    St = g_dnn_bin(train_data, val_data, test_data, t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    S_R_X_ibs = St['S_R_X_untreat']
    S_R_X_ibs1 = St['S_R_X_treated']
    Lambda_R_X_untreat = St['Lambda_R_X_untreat']
    Lambda_R_X_treated = St['Lambda_R_X_treated']
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bined(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.07497999999999999
0.015199986842099568
0.0067379459143510345
0.0035988400938544176


In [35]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Deep_ed
import torch
from log_hazard_binary import g_dnn_bin
from optITR import optITR_deep_bin
import time
#As an example for linear case
patiences = 30
n_node = 64
n_layer = 2
t_nodes = 100
t_fig = 50
n_lr = 1e-3
n_epoch = 200
s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Deep_ed(400, 0.5)
    train_data = gendata_Deep_ed(1600, 0.5)
    St = g_dnn_bin(train_data, val_data, test_data, t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    S_R_X_ibs = St['S_R_X_untreat']
    S_R_X_ibs1 = St['S_R_X_treated']
    Lambda_R_X_untreat = St['Lambda_R_X_untreat']
    Lambda_R_X_treated = St['Lambda_R_X_treated']
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.14215
0.02312828355066584
0.029863839035985357
0.007938674725355314
